# SGD with bias

In [1]:
import os

import numpy as np
import pandas as pd
from subprocess import call
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# change default figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

## MovieLens Dataset

In [4]:
file_dir = os.path.join('data', 'ml-100k')
file_path = os.path.join(file_dir, 'u.data')

if not os.path.isdir(file_dir):
    call(['curl', '-O', 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'])
    call(['unzip', 'ml-100k.zip'])
    
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(file_path, sep = '\t', names=names)
print(df.shape)
df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
# create the rating matrix r_{ui}
# remember to subtract the user and item id by 1 since the indices starts from 0
n_users = df['user_id'].unique().shape[0]
n_items = df['item_id'].unique().shape[0]
# initialize
ratings = np.zeros((n_users, n_items))
for row in df.itertuples(index=False): # iterrowよりも高速
    ratings[row.user_id-1, row.item_id-1] = row.rating

# compute the no-zero elements in the rating matrix
matrix_size = np.prod(ratings.shape)
interaction = np.flatnonzero(ratings).shape[0] # retrun "indices" tha are non-zero
sparsity = (interaction/matrix_size) * 100

print('dimensition: ', ratings.shape)
print('sparsity: {:.1f}%'.format(sparsity))
ratings

dimensition:  (943, 1682)
sparsity: 6.3%


array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

## Matrix Factorization with SGD

With SGD, we again take derivatives of the loss function, but we take the derivative with respect to each variable in the model.  
The “stochastic” aspect of the algorithm involves taking the derivative and updating feature weights one individual sample at a time.  
So, for each sample, we take the derivative of each variable, set them all equal to zero, solve for the feature weights, and update each feature. Somehow this method actually converges.

We will use a similar loss function to before, but I am going to add some more details to the model.  
Instead of assuming that a user u’s rating for item i can be described simply by the dot product of the user and item latent vectors, we will consider that each user and item can have a bias term associated with them. 
The rational is that certan users might tend to rate all movies highly, or certain movies may tend to always have low ratings.  
The way that I think about it is that the bias term takes care of the “DC” part of the signal which allows the latent factors to account for the more detailed variance in signal (kind of like the AC part).  
We will also include a global bias term as well. With all things combined, our predicted rating becomes

$$\hat r_{ui} = \mu + \vec{b_u} + \vec{b_i} + \vec{x_u}^T \cdot \vec{y_i}$$

where $\mu$ is the global bias, and $\vec{b_u}(\vec{b_i})$ is the user (item) bias.  
Our loss function now becomes

$$L = \sum_{u,i}(\vec{r_{ui}} - (\mu + \vec{b_u} + \vec{b_i} + \vec{x_u}^T \cdot \vec{y_i}))^2 + \lambda_{xb}\sum_u|\vec{b_u}|^2 + \lambda_{yb}\sum_i|\vec{b_i}|^2 + \lambda_{xf}\sum_u|\vec{x_u}|^2 + \lambda_{yf}\sum_i|\vec{y_i}|^2$$

where we have added on extra bias regularization terms.  
We want to update each feature (user and item latent factors and bias terms) with each sample.  
The update for the user bias is given by

$$\vec{b_u}\gets\vec{b_u} - \eta\frac{\partial L}{\partial \vec{b_u}}$$

where $\eta$ is the learning rate which weights how much our update modifies the feature weights.  
The derivative term is given by

$$\frac{\partial L}{\partial \vec{b_u}} = 2(\vec{r_{ui}} - (\mu + \vec{b_u} +\vec{b_i} + \vec{x_u}^T \cdot \vec{y_i}))(-1) + 2\lambda_{xb}\vec{b_u}$$
$$\frac{\partial L}{\partial \vec{b_u}} = 2(\vec{e_{ui}})(-1) + 2\lambda_{xb}\vec{b_u}$$
$$\frac{\partial L}{\partial \vec{b_u}} = -\vec{e_{ui}} + \lambda_{xb}\vec{b_u}$$

where **$e_{ui}$ represents the error in our prediction** and we have dropped the factor of 2 (we can assume it gets rolled up in the learning rate). For all of our features, the updates end up being

$$\vec{b_u}\gets\vec{b_u} + \eta(\vec{e_{ui}} - \lambda_{xb}\vec{b_u})$$
$$\vec{b_i}\gets\vec{b_i} + \eta(\vec{e_{ui}} - \lambda_{yb}\vec{b_i})$$
$$\vec{x_u}\gets\vec{x_u} + \eta(\vec{e_{ui}}\vec{y_i} - \lambda_{xf}\vec{x_u})$$
$$\vec{y_i}\gets\vec{y_i} + \eta(\vec{e_{ui}}\vec{x_u} - \lambda_{yf}\vec{y_i})$$

In [10]:
class ExplictMF():
    """
    Train a matrix factorization mode to predict empty entries in a matrix.
    Te terminology assumes a ratings matrix which is ~ user x item
    
    Parameters
    ----------------
    ratings : ndarray
        user x Item matrix with corresponding ratings
    
    n_factors : int
        number of latent factors to use in matrix
        factorization model, some machine-learning libraries denote this as 'rank'
        
    learnig : str
        method of optimization. 
        options include 'sgd' of 'als'.
        
    item_fact_reg : float
        regularization term for item latent factors
        
    user_fact_reg : float
        regularization term for user latent factors
        
    item_bias_reg : float
        regularization term for item biases
        
    user_bias_reg : float
        regularization term for user biases
        
    verbose : bool
        whether or not to printout training progress
    """
    
    def __init__(self, ratings, n_factors=40, learning='sgd',
                     item_fact_reg=0.0, user_fact_reg=0.0, item_bias_reg=0.0, user_bias_reg=0.0,
                    verbose=False):
        self.ratings = ratings
        self.n_users, self.n_items = rating.shape
        self.n_factors = n_factors
        self.item_fact_reg = item_fact_reg
        self.user_fact_reg = user_fact_reg
        self.item_bias_reg = item_bias_reg
        self.user_bias_reg = user_bias_reg
        self.learning = learnig
        
        if self.learning == 'sgd':
            self.sample_row, self.sample_col = self.ratings.nonzero() # nonzeroのindexを返す
            self.n_samples = len(self.sample_row)
        self._v = verbose
        
        
    def train(self, n_iter=10, learning_rate=0.1):
        # initrialize model for n_iter iterations from scratch
        self.user_vecs = np.random.normal(scale=1./self.n_factors, size=(self.n_users, self.n_factors))
        self.item_vecs = np.random.normal(scale=1./self.n_factors, size=(self.n_items, self.n_factors))
        
        self.learning_rate = learning_rate
        self.user_bias = np.zeros(self.n_users)
        self.item_bias = np.zeros(self.n_items)
        self.global_bias = np.mean(self.ratings[np.where(self.ratings != 0)])
        self.partial_train(n_iter)
        
        
    def partial_train(self, n_iter):
        """
        Train model for n_iter iterations.
        Can be called multiple times for further training.
        """
        self.test_mse_record = []
        self.train_mse_record = []
        ctr = 1
        while ctr <= n_iter:
            if ctr % 10 == 0 and self._v:
                print('\tcurrent iteration: {}'.format(ctr))
            
            self.training_indices = np.arrange(self.n_samples)
            np.random.shuffle(self.training_indices)
            self.sgd()
            
            ctr += 1
        
        
    def sgd(self):
        for idx in self.training_indices:
            u = self.sample_row[idx]
            i = self.sample_col[idx]
            prediction = self.predict(u, i)
            
            e = (self.ratings[u, i] - prediction) # error
            
            # Update bias
            self.user_bias[u] += self.learning_rate * (e - self.user_bias_reg * self.user_bias[u])
            self.item_bias[i] += self.learning_rate * (e - self.item_bias_reg * self.item_bias[i])
            
            # Update latent factors
            self.user_vecs[u, :] += self.learning_rate * (e *  self.item_vecs[i, :] - self.user_fact_reg * self.user_vecs[u, :])
            self.item_vecs[i :] += self.learning_rate * (e *  self.user_vecs[u, :] - self.item_fact_reg * self.item_vecs[i, :])
            
            
    def predict(self, u, i):
        """
        SIngle user and item prediction.
        """
        prediction = self.global_bias + self.user_bias[u] + self.item_bias[i] + self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
        return prediction
    
    
    @staticmethod
    def compute_mse(y_true, y_pred):
        """
        ignore zero terms prior to comparing the mse
        """
        mask = np.nonzero(y_true)
        mse = mean_squared_error(y_true[mask], y_pred[mask])
        return mse

In [11]:
def plot_learning_curve(model):
    """
    visualize the training/testing loss
    """
    linewidth = 3
    plt.plot(model.test_mse_record, label='Test', linewidth=linewidth)
    plt.plot(model.train_mse_record, label='Train', linewidth=linewidth)
    plt.xlabel('iterations')
    plt.ylabel('MSE')
    plt.legend(loc='best')